### data2의 피처들을 활용하여 홍보문구, 연관 키워드 생성 후 data3에 저장

1. data3.csv
    - product_title : 생성한 상품 제목
    - detailed_description : 생성한 홍보문구
    - related_search_keywords : 생성한 연관 키워드 리스트
2. prompts 디렉토리에 프롬프트 텍스트 저장되어 있음

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pandas as pd

In [9]:
llm = ChatOpenAI(model='gpt-3-turbo-0125')

with open("prompts/product_title_prompt.txt", 'r', encoding='utf-8') as file: product_title_prompt = file.read()
with open("prompts/product_description_prompt.txt", 'r', encoding='utf-8') as file: product_description_prompt = file.read()
with open("prompts/related_keywords_prompt.txt", 'r', encoding='utf-8') as file: related_keywords_prompt = file.read()

input_prompt_text = "Rough description: {product_name} \n" \
              "Category: {product_category} \n" \
              "Selling points: {product_selling_points} \n"

prompt1 = ChatPromptTemplate.from_template(product_title_prompt + input_prompt_text)
prompt2 = ChatPromptTemplate.from_template(product_description_prompt + input_prompt_text)
prompt3 = ChatPromptTemplate.from_template(related_keywords_prompt + input_prompt_text)

parser = StrOutputParser()


chain1 = {"product_category": RunnablePassthrough(), "product_name": RunnablePassthrough(), "product_selling_points": RunnablePassthrough()} | prompt1 | llm | parser
chain2 = {"product_category": RunnablePassthrough(), "product_name": RunnablePassthrough(), "product_selling_points": RunnablePassthrough()} | prompt2 | llm | parser
chain3 = {"product_category": RunnablePassthrough(), "product_name": RunnablePassthrough(), "product_selling_points": RunnablePassthrough()} | prompt3 | llm | parser

In [ ]:
data2_csv = pd.read_csv("data/data2.csv")
product_name_chn = data2_csv["product_name_chn"]
product_name_kor = data2_csv["product_name_kor"]
product_category = data2_csv["product_category"]
product_selling_points = data2_csv["product_selling_points"]


product_title = chain1.batch(list(zip(product_name_kor, product_category, product_selling_points)), config={"max_concurrency": 200})
detailed_description = chain2.batch(list(zip(product_name_kor, product_category, product_selling_points)), config={"max_concurrency": 200})
related_keywords = chain3.batch(list(zip(product_name_kor, product_category, product_selling_points)), config={"max_concurrency": 200})

In [ ]:
for i in range(len(product_title)):
    if (product_title[i][:14] == "Product title:"): product_title[i] = product_title[i][15:]

for j in range(len(detailed_description)):
    if (detailed_description[j][:33] == "Detailed promotional description:"): detailed_description[j] = detailed_description[j][35:]

for k in range(len(related_keywords)):
    if ("Related search keywords:" in related_keywords[k]):
        related_keywords[k] = related_keywords[k].split("Related search keywords: ")[1][:-1]

In [ ]:
df = pd.DataFrame({'product_name_chn': product_name_chn,
                   'product_name_kor': product_name_kor,
                   'product_title' : product_title,
                   'detailed_description': detailed_description,
                   'related_search_keywords': related_keywords})
df.to_csv("data/data3.csv", index=False)